# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [15]:
import torch
import os
from ocpmodels.datasets import SinglePointLmdbDataset, TrajectoryLmdbDataset
from ocpmodels.trainers import ForcesTrainer, EnergyTrainer
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

In [6]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

False


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Dataset

This examples uses the LMDB generated from the following [tutorial](http://laikapack.cheme.cmu.edu/notebook/open-catalyst-project/mshuaibi/notebooks/projects/ocp/docs/source/tutorials/lmdb_dataset_creation.ipynb). Please run that notebook before moving on. Alternatively, if you have other LMDBs available you may specify that instead.

In [7]:
# set the path to your local lmdb directory
train_src = "/Users/ssharlin/Desktop/Fall 2021/CMSC 678/ML project/is2res_train_val_test_lmdbs/data/is2re/10k/train/data.lmdb"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [17]:
task = {
    'dataset': 'single_point_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [26]:
# model = {
#     'name': 'schnet',
#     'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
#     'num_filters': 256,
#     'num_interactions': 3,
#     'num_gaussians': 200,
#     'cutoff': 6.0
# }

model = {
  'name': 'schnet',
  'hidden_channels': 256,
  'num_filters': 128,
  'num_interactions': 3,
  'num_gaussians': 100,
  'cutoff': 6.0,
  'use_pbc': True,
  'regress_forces': False,
}

**Optimizer**

In [23]:
# optimizer = {
#     'batch_size': 16, # if hitting GPU memory issues, lower this
#     'eval_batch_size': 8,
#     'num_workers': 0,
#     'lr_initial': 0.0001,
#     'scheduler': "ReduceLROnPlateau",
#     'mode': "min",
#     'factor': 0.8,
#     'patience': 3,
#     'max_epochs': 80,
#     'max_epochs': 1, # used for demonstration purposes
#     'force_coefficient': 100,
# }


optimizer = {
  'batch_size': 64,
  'eval_batch_size': 64,
  'num_workers': 0,
  'lr_initial': 0.005,
  'lr_gamma': 0.1,
  'lr_milestones': [1562, 2343, 3125],
  'warmup_steps': 468,
  'warmup_factor': 0.2,
  'max_epochs': 30,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [24]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [27]:
trainer = EnergyTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=5,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-12-17-14-33-20-SchNet-example
  commit: null
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-12-17-14-33-20-SchNet-example
  print_every: 5
  results_dir: ./results/2021-12-17-14-33-20-SchNet-example
  seed: 0
  timestamp_id: 2021-12-17-14-33-20-SchNet-example
dataset:
  normalize_labels: false
  src: /Users/ssharlin/Desktop/Fall 2021/CMSC 678/ML project/is2res_train_val_test_lmdbs/data/is2re/10k/train/data.lmdb
gpus: 0
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 256
  num_filters: 128
  num_gaussians: 100
  num_interactions: 3
  regress_forces: false
  use_pbc: true
optim:
  batch_size: 64
  eval_batch_size: 64
  lr_gamma: 0.1
  lr_initial: 0.005
  lr_milestones:
  - 1562
  - 2343
  - 3125
  max_epochs: 30
  num_workers: 0
  warmup_factor: 0.2
  warmup_steps: 468
slurm: {}
task:
  dataset: single_point_lmdb
  description: Regressing to energies and forces for DFT trajectories

fatal: Not a valid object name HEAD
2021-12-17 14:33:44 (WARNING): Model gradient logging to tensorboard not yet supported.


## Check the model

In [28]:
print(trainer.model)

OCPDataParallel(
  (module): SchNetWrap(hidden_channels=256, num_filters=128, num_interactions=3, num_gaussians=100, cutoff=6.0)
)


## Train

In [29]:
trainer.train()

/Users/ssharlin/miniforge3/envs/ocp-models/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


energy_mae: 3.73e+01, energy_mse: 3.02e+03, energy_within_threshold: 0.00e+00, loss: 3.73e+01, lr: 1.03e-03, epoch: 3.18e-02, step: 5.00e+00
energy_mae: 2.43e+01, energy_mse: 1.05e+03, energy_within_threshold: 1.56e-02, loss: 2.43e+01, lr: 1.08e-03, epoch: 6.37e-02, step: 1.00e+01
energy_mae: 2.13e+01, energy_mse: 8.09e+02, energy_within_threshold: 0.00e+00, loss: 2.13e+01, lr: 1.12e-03, epoch: 9.55e-02, step: 1.50e+01
energy_mae: 1.67e+01, energy_mse: 5.79e+02, energy_within_threshold: 0.00e+00, loss: 1.67e+01, lr: 1.16e-03, epoch: 1.27e-01, step: 2.00e+01
energy_mae: 1.83e+01, energy_mse: 6.82e+02, energy_within_threshold: 0.00e+00, loss: 1.83e+01, lr: 1.21e-03, epoch: 1.59e-01, step: 2.50e+01
energy_mae: 1.54e+01, energy_mse: 7.37e+02, energy_within_threshold: 0.00e+00, loss: 1.54e+01, lr: 1.25e-03, epoch: 1.91e-01, step: 3.00e+01
energy_mae: 1.52e+01, energy_mse: 4.10e+02, energy_within_threshold: 0.00e+00, loss: 1.52e+01, lr: 1.29e-03, epoch: 2.23e-01, step: 3.50e+01
energy_mae: 1

device 0: 100%|███████████████████████████████| 157/157 [01:57<00:00,  1.33it/s]

2021-12-17 14:39:54 (INFO): energy_mae: 8.4726, energy_mse: 155.0149, energy_within_threshold: 0.0017, loss: 8.4780, epoch: 1.0000
2021-12-17 14:39:54 (INFO): Predicting on test.



device 0: 100%|███████████████████████████████| 157/157 [01:59<00:00,  1.32it/s]


...


device 0: 100%|███████████████████████████████| 157/157 [02:53<00:00,  1.11s/it]

2021-12-17 19:19:20 (INFO): Writing results to ./results/2021-12-17-14-33-20-SchNet-example/is2re_predictions.npz


energy_mae: 6.94e-01, energy_mse: 1.07e+00, energy_within_threshold: 1.56e-02, loss: 6.94e-01, lr: 5.00e-06, epoch: 2.90e+01, step: 4.56e+03
energy_mae: 5.94e-01, energy_mse: 7.31e-01, energy_within_threshold: 3.12e-02, loss: 5.94e-01, lr: 5.00e-06, epoch: 2.90e+01, step: 4.56e+03
energy_mae: 9.17e-01, energy_mse: 2.05e+00, energy_within_threshold: 1.56e-02, loss: 9.17e-01, lr: 5.00e-06, epoch: 2.91e+01, step: 4.56e+03
energy_mae: 8.01e-01, energy_mse: 1.28e+00, energy_within_threshold: 4.69e-02, loss: 8.01e-01, lr: 5.00e-06, epoch: 2.91e+01, step: 4.57e+03
energy_mae: 7.50e-01, energy_mse: 1.15e+00, energy_within_threshold: 3.12e-02, loss: 7.50e-01, lr: 5.00e-06, epoch: 2.91e+01, step: 4.58e+03
energy_mae: 7.53e-01, energy_mse: 1.25e+00, energy_within_threshold: 4.69e-02, loss: 7.53e-01, lr: 5.00e-06, epoch: 2.92e+01, step: 4.58e+03
energy_mae: 8.70e-01, energy_mse: 1.60e+00, energy_within_threshold: 3.12e-02, loss: 8.70e-01, lr: 5.00e-06, epoch: 2.92e+01, step: 4.58e+03
energy_mae: 9

device 0: 100%|███████████████████████████████| 157/157 [03:17<00:00,  1.26s/it]

2021-12-17 19:29:22 (INFO): energy_mae: 0.8264, energy_mse: 1.5034, energy_within_threshold: 0.0204, loss: 0.8258, epoch: 30.0000


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir`):

In [32]:
checkpoint_path = os.path.join(trainer.config["cmd"]["checkpoint_dir"], "checkpoint.pt")
checkpoint_path

'./checkpoints/2021-12-17-14-33-20-SchNet-example/checkpoint.pt'

In [36]:
model = {
    'name': 'schnet',
    'hidden_channels': 256, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 128,
    'num_interactions': 3,
    'num_gaussians': 100,
    'cutoff': 6.0
}


pretrained_trainer = EnergyTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_checkpoint(checkpoint_path=checkpoint_path)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-12-17-22-39-44-SchNet-example
  commit: null
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-12-17-22-39-44-SchNet-example
  print_every: 10
  results_dir: ./results/2021-12-17-22-39-44-SchNet-example
  seed: 0
  timestamp_id: 2021-12-17-22-39-44-SchNet-example
dataset:
  normalize_labels: false
  src: /Users/ssharlin/Desktop/Fall 2021/CMSC 678/ML project/is2res_train_val_test_lmdbs/data/is2re/10k/train/data.lmdb
gpus: 0
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 256
  num_filters: 128
  num_gaussians: 100
  num_interactions: 3
optim:
  batch_size: 64
  eval_batch_size: 64
  lr_gamma: 0.1
  lr_initial: 0.005
  lr_milestones:
  - 1562
  - 2343
  - 3125
  max_epochs: 30
  num_workers: 0
  warmup_factor: 0.2
  warmup_steps: 468
slurm: {}
task:
  dataset: single_point_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  

fatal: Not a valid object name HEAD
2021-12-17 22:39:55 (WARNING): Model gradient logging to tensorboard not yet supported.


2021-12-17 22:39:55 (INFO): Loading checkpoint from: ./checkpoints/2021-12-17-14-33-20-SchNet-example/checkpoint.pt


## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.